In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import requests

In [2]:
import pickle


def load_pickle(path):
    with open(path, 'rb') as pickle_file:
        return pickle.load(pickle_file)

# Load amc_project_dict
amc_project_dict = load_pickle('pickle/amc_project_dict.pickle')

project_policy_dict = load_pickle('pickle/project_policy_dict.pickle')
project_performance_dict = load_pickle('pickle/project_performance_dict.pickle')
project_investment_dict = load_pickle('pickle/project_investment_dict.pickle')
project_benchmark_dict = load_pickle('pickle/project_benchmark_dict.pickle')
project_5YearLost_dict = load_pickle('pickle/project_5YearLost_dict.pickle')
project_nav_dict = load_pickle('pickle/project_nav_dict2.pickle')
project_nav_pm_dict = load_pickle('pickle/project_nav_pm_dict2.pickle')
project_fund_full_port_dict = load_pickle('pickle/project_fund_full_port_dict2.pickle')
project_fund_top5_dict = load_pickle('pickle/project_fund_top5_dict2.pickle')

In [41]:
SUBSCRIPTION_KEY = '886c69c66f8f4f2b9159a2877cb4dca4'

# Request headers
headers = {
    'Ocp-Apim-Subscription-Key': SUBSCRIPTION_KEY,
}

amc_url = 'https://api.sec.or.th/FundFactsheet/fund/amc'

res = requests.get(amc_url, headers=headers)
amc_list = res.json()

amc_df_dict = {
    'unique_id': [item['unique_id'] for item in amc_list],
    'name_th': [item['name_th'] for item in amc_list],
    'name_en': [item['name_en'] for item in amc_list],
    'last_upd_date': [item['last_upd_date'] for item in amc_list],
}
amc_df = pd.DataFrame(amc_df_dict)
amc_df = amc_df.set_index('unique_id')
amc_df.head()

,name_th,name_en,last_upd_date
unique_id,,,
C0000000021,บริษัท หลักทรัพย์จัดการกองทุนกสิกรไทย จำกัด,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,2020-04-09T05:04:20
C0000000023,บริษัท หลักทรัพย์จัดการกองทุนเอ็มเอฟซี จำกัด (...,MFC ASSET MANAGEMENT PUBLIC COMPANY LIMITED,2020-04-09T05:04:20
C0000000124,บริษัท หลักทรัพย์จัดการกองทุนเมอร์ชั่น พาร์ทเน...,MERCHANT PARTNERS ASSET MANAGEMENT LIMITED,2020-04-09T05:04:20
C0000000182,บริษัท หลักทรัพย์จัดการกองทุนทหารไทย จำกัด,TMB ASSET MANAGEMENT COMPANY LIMITED,2020-04-09T05:04:20
C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,SCB ASSET MANAGEMENT COMPANY LIMITED,2020-04-09T05:04:20


In [42]:
amc_project_df_dict = {
    'amc_id': [],
    'amc_name': [],
    'project_id': [],
    'project_name': [],
    'fund_status':[],
    'management_style': [],
    'policy_desc': []
}

for amc_id, project_list in tqdm(amc_project_dict.items()):
    amc = amc_df.loc[amc_id]
    amc_name = amc['name_en']
    for project in project_list:
        project_id = project['proj_id']
        project_name = project['proj_name_en']
        fund_status=project['fund_status']

        amc_project_df_dict.get('amc_id').append(amc_id)
        amc_project_df_dict.get('amc_name').append(amc_name)
        amc_project_df_dict.get('project_id').append(project_id)
        amc_project_df_dict.get('project_name').append(project_name)
        amc_project_df_dict.get('fund_status').append(fund_status)

        management_style = project_policy_dict[project_id]['management_style']

        amc_project_df_dict.get('management_style').append(management_style)
        
        policy_desc = project_policy_dict[project_id]['policy_desc']

        amc_project_df_dict.get('policy_desc').append(policy_desc)

amc_project_df = pd.DataFrame(amc_project_df_dict)
amc_project_df.head()

,amc_id,amc_name,project_id,project_name,fund_status,management_style,policy_desc
0,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0000_2553,K Fixed Income 3 Months B Fund,LI,-,ตราสารแห่งหนี้
1,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0001_2558,K Enhanced Foreign Fixed Income 1 Year AD Fund,LI,-,ตราสารแห่งหนี้
2,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0003_2558,K Enhanced Fixed Income 3 Months K Fund,LI,-,ตราสารแห่งหนี้
3,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0003_2563,K Foreign Fixed Income 6 Months HX Fund,EX,BH,ตราสารหนี้
4,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0004_2555,K Fixed Income 6 Months AJ Fund,LI,-,ตราสารแห่งหนี้


In [43]:
((amc_project_df.management_style == 'PM') & ( amc_project_df.fund_status == 'RG') & (amc_project_df.policy_desc == 'ตราสารทุน')).value_counts()

False    11023
True        45
dtype: int64

In [44]:
df_dict = {
    'project_id': [],
    'loss_five_year_percent': [],
    'sd_five_year_percent': [],
    'tracking_error_percent': [],
}

amc_project_df = amc_project_df.set_index('project_id')

for k, v in project_5YearLost_dict.items():
    project_id = k
    perf_list = v
    amc = amc_project_df.loc[project_id]
    amc_id = amc['amc_id']
    amc_name = amc['amc_name']
    project_name = amc['project_name']
    if v is None:
        continue
    v = v[0]
#         df_dict.get('amc_id').append(amc_id)
#         df_dict.get('amc_name').append(amc_name)
    df_dict.get('project_id').append(project_id)
#         df_dict.get('project_name').append(project_name)
    df_dict.get('loss_five_year_percent').append(v['loss_five_year_percent'])
    df_dict.get('sd_five_year_percent').append(v['sd_five_year_percent'])
    df_dict.get('tracking_error_percent').append(v['tracking_error_percent'])
        
amc_project_df = amc_project_df.reset_index()

project_5YearLost_dict = pd.DataFrame(df_dict)
project_5YearLost_dict.to_csv('csv/project_5YearLost.csv', index=False)
project_5YearLost_dict

,project_id,loss_five_year_percent,sd_five_year_percent,tracking_error_percent
0,M0008_2537,-15.18,12.32,-
1,M0009_2543,-18.44,11.91,-
2,M0014_2536,-18.51,12.44,-
3,M0023_2538,-22.81,11.09,-
4,M0027_2535,-17.87,12.05,-
...,...,...,...,...
578,M0177_2551,-37.06,16.93,0.92
579,M0066_2562,-18.39,12.72,0.82
580,M0259_2560,-13.34,11.74,0.0829
581,M0356_2561,-5.68,9.81,0.0790


In [45]:
project_performance_df_dict = {
    'amc_id': [],
#     'amc_name': [],
    'project_id': [],
#     'project_name': [],
    'class_abbr_name': [],
    'reference_period': [],
    'performance_type_desc': [],
    'performance_val': [],
    'as_of_date': [],
}

amc_project_df = amc_project_df.set_index('project_id')

for k, v in project_performance_dict.items():
    project_id = k
    perf_list = v
    amc = amc_project_df.loc[project_id]
    amc_id = amc['amc_id']
    amc_name = amc['amc_name']
    project_name = amc['project_name']
    for v in perf_list:
        if v is None:
            continue
        project_performance_df_dict.get('amc_id').append(amc_id)
#         project_performance_df_dict.get('amc_name').append(amc_name)
        project_performance_df_dict.get('project_id').append(project_id)
#         project_performance_df_dict.get('project_name').append(project_name)
        project_performance_df_dict.get('class_abbr_name').append(v['class_abbr_name'])
        project_performance_df_dict.get('reference_period').append(v['reference_period'])
        project_performance_df_dict.get('performance_type_desc').append(v['performance_type_desc'])
        project_performance_df_dict.get('performance_val').append(v['performance_val'])
        project_performance_df_dict.get('as_of_date').append(v['as_of_date'])
        
amc_project_df = amc_project_df.reset_index()

project_performance_df = pd.DataFrame(project_performance_df_dict)
project_performance_df.to_csv('csv/project_performance.csv', index=False)
project_performance_df

,amc_id,project_id,class_abbr_name,reference_period,performance_type_desc,performance_val,as_of_date
0,C0000000021,M0008_2537,-,1_year,ความผันผวนของกองทุนรวม,10.4000,30 ธันวาคม 2562
1,C0000000021,M0008_2537,-,1_year_percentile,ความผันผวนของกองทุนรวม,95.0000,30 ธันวาคม 2562
2,C0000000021,M0008_2537,-,10_year,ความผันผวนของกองทุนรวม,16.6800,30 ธันวาคม 2562
3,C0000000021,M0008_2537,-,10_year_percentile,ความผันผวนของกองทุนรวม,100.0000,30 ธันวาคม 2562
4,C0000000021,M0008_2537,-,3_month,ความผันผวนของกองทุนรวม,11.8600,30 ธันวาคม 2562
...,...,...,...,...,...,...,...
32508,C0000021747,M0594_2560,-,3_month_percentile,ผลตอบแทนตัวชี้วัด,75 - 95,30 ธันวาคม 2562
32509,C0000021747,M0594_2560,-,6_month,ผลตอบแทนตัวชี้วัด,-11.56,30 ธันวาคม 2562
32510,C0000021747,M0594_2560,-,6_month_percentile,ผลตอบแทนตัวชี้วัด,75 - 95,30 ธันวาคม 2562
32511,C0000021747,M0594_2560,-,year_to_date,ผลตอบแทนตัวชี้วัด,2.25,30 ธันวาคม 2562


In [46]:
project_performance_pivot = project_performance_df.pivot_table(
    index=['project_id', 'as_of_date', 'reference_period'],
    columns='performance_type_desc',
    values=['performance_val'],
    aggfunc='first'
).reset_index()

project_performance_pivot.to_csv('csv/project_performance_pivot.csv')
project_performance_pivot

project_id        as_of_date reference_period  \
performance_type_desc                                                  
0                      M0000_2552  28 มิถุนายน 2562          10_year   
1                      M0000_2552  28 มิถุนายน 2562           1_year   
2                      M0000_2552  28 มิถุนายน 2562          3_month   
3                      M0000_2552  28 มิถุนายน 2562           3_year   
4                      M0000_2552  28 มิถุนายน 2562           5_year   
...                           ...               ...              ...   
7318                   M1242_2557   30 ธันวาคม 2562          3_month   
7319                   M1242_2557   30 ธันวาคม 2562           3_year   
7320                   M1242_2557   30 ธันวาคม 2562          6_month   
7321                   M1242_2557   30 ธันวาคม 2562     year_to_date   
7322                   M1242_2557   30 ธันวาคม 2562   ตั้งแต่จัดตั้ง   

                             performance_val                         \
performance_type_desc ความผันผวนของกองทุนรวม ความผันผวนของตัวชี้วัด   
0                                      15.15                  15.74   
1                                       9.72                  10.89   
2                                       7.71                   7.76   
3                                      10.16                  10.42   
4                                      11.18                  11.64   
...                                      ...                    ...   
7318                                    8.43                   4.43   
7319                                   13.61                  24.20   
7320                                   10.32                   9.55   
7321                                   12.55                   8.16   
7322                                   15.54                  20.11   

                                                           
performance_type_desc ผลตอบแทนกองทุนรวม ผลตอบแทนตัวชี้วัด  
0                             15.131806             15.22  
1                              7.212548             11.86  
2                              4.998833              6.61  
3                              8.631391              9.54  
4                              6.097008              6.38  
...                                 ...               ...  
7318                               6.99              1.28  
7319                               3.52             -1.23  
7320                              10.51              1.91  
7321                              14.37              1.36  
7322                              -1.34              1.82  

[7323 rows x 7 columns]

In [47]:
PM_SET50_proj_ids = []

amc_project_df = amc_project_df.set_index('project_id')

for key, value in tqdm(project_benchmark_dict.items()):
    project_id = key
    bm_list = value
    amc = amc_project_df.loc[project_id]
    policy_desc = amc['policy_desc']
    fund_status = amc['fund_status']
    management_style = amc['management_style']
    
#     if management_style == 'PM' and fund_status == 'RG' and policy_desc == 'ตราสารทุน':
    if management_style == 'PM' and fund_status == 'RG':
        for bm in bm_list:
            if bm['benchmark'] == 'ดัชนีผลตอบแทนรวม SET 50 (SET50 TRI)':
                PM_SET50_proj_ids.append(project_id)
                break

amc_project_df = amc_project_df.reset_index()

PM_SET50_proj_ids

['M0425_2559',
 'M0575_2560',
 'M0580_2560',
 'M0653_2559',
 'M0691_2559',
 'M0259_2550',
 'M0227_2562',
 'M0292_2562',
 'M0001_2552',
 'M0209_2548',
 'M0738_2556',
 'M0155_2547',
 'M0597_2552',
 'M0787_2554',
 'M0132_2557',
 'M0712_2547',
 'M0943_2554']

In [48]:
project_nav_df_dict = {
    'amc_id': [],
    'amc_name': [],
    'project_id': [],
    'project_name': [],
    'management_style': [],
    'policy_desc': [],
    'fund_status': [],
    'nav_date': [],
    'net_asset': [],
    'last_val': [],
    'previous_val': [],
}

amc_project_df = amc_project_df.set_index('project_id')

for k, v in project_nav_dict.items():
    project_id = k
    date_dict = v
    amc = amc_project_df.loc[project_id]
    amc_id = amc['amc_id']
    amc_name = amc['amc_name']
    project_name = amc['project_name']
    
    management_style = amc['management_style']
    policy_desc = amc['policy_desc']
    fund_status = amc['fund_status']
    if management_style == 'PM':
        if project_id not in PM_SET50_proj_ids:
            continue
            
    for k, v in date_dict.items():
        date = k
        if v is None:
            continue
        project_nav_df_dict.get('amc_id').append(amc_id)
        project_nav_df_dict.get('amc_name').append(amc_name)
        project_nav_df_dict.get('project_id').append(project_id)
        project_nav_df_dict.get('project_name').append(project_name)
        project_nav_df_dict.get('management_style').append(management_style)
        project_nav_df_dict.get('policy_desc').append(policy_desc)
        project_nav_df_dict.get('fund_status').append(fund_status)
        project_nav_df_dict.get('nav_date').append(date)
        project_nav_df_dict.get('net_asset').append(v['net_asset'])
        project_nav_df_dict.get('last_val').append(v['last_val'])
        project_nav_df_dict.get('previous_val').append(v['previous_val'])
        
amc_project_df = amc_project_df.reset_index()

project_nav_df = pd.DataFrame(project_nav_df_dict)
project_nav_df.to_csv('csv/project_nav.csv', index=False)
print(project_nav_df[project_nav_df['management_style'] == 'AM']['project_id'].unique().shape)
project_nav_df

(662,)


,amc_id,amc_name,project_id,project_name,management_style,policy_desc,fund_status,nav_date,net_asset,last_val,previous_val
0,C0000000714,THANACHART FUND MANAGEMENT COMPANY LIMITED,M0913_2554,Thanachart Low Beta Fund,AM,ตราสารแห่งทุน,RG,2020-01-20,4.820166e+09,21.8660,21.9716
1,C0000000714,THANACHART FUND MANAGEMENT COMPANY LIMITED,M0913_2554,Thanachart Low Beta Fund,AM,ตราสารแห่งทุน,RG,2020-01-14,4.833400e+09,21.8799,21.8416
2,C0000000714,THANACHART FUND MANAGEMENT COMPANY LIMITED,M0913_2554,Thanachart Low Beta Fund,AM,ตราสารแห่งทุน,RG,2020-01-17,4.843628e+09,21.9716,21.8791
3,C0000000714,THANACHART FUND MANAGEMENT COMPANY LIMITED,M0913_2554,Thanachart Low Beta Fund,AM,ตราสารแห่งทุน,RG,2020-01-06,4.737245e+09,21.3101,21.6317
4,C0000000714,THANACHART FUND MANAGEMENT COMPANY LIMITED,M0913_2554,Thanachart Low Beta Fund,AM,ตราสารแห่งทุน,RG,2020-01-21,4.770255e+09,21.6478,21.8660
...,...,...,...,...,...,...,...,...,...,...,...
37481,C0000000023,MFC ASSET MANAGEMENT PUBLIC COMPANY LIMITED,M0210_2561,MFC Multi Asset Fund,AM,ผสม,RG,2020-03-09,3.365123e+08,9.5609,9.8728
37482,C0000000023,MFC ASSET MANAGEMENT PUBLIC COMPANY LIMITED,M0210_2561,MFC Multi Asset Fund,AM,ผสม,RG,2020-03-12,3.150892e+08,8.9785,9.4955
37483,C0000000023,MFC ASSET MANAGEMENT PUBLIC COMPANY LIMITED,M0210_2561,MFC Multi Asset Fund,AM,ผสม,RG,2020-03-13,3.244112e+08,9.2444,8.9785
37484,C0000000023,MFC ASSET MANAGEMENT PUBLIC COMPANY LIMITED,M0210_2561,MFC Multi Asset Fund,AM,ผสม,RG,2020-03-20,2.913679e+08,8.4860,8.5704


In [49]:
project_nav_df_dict = {
    'amc_id': [],
    'amc_name': [],
    'project_id': [],
    'project_name': [],
    'management_style': [],
    'policy_desc': [],
    'fund_status': [],
    'nav_date': [],
    'net_asset': [],
    'last_val': [],
    'previous_val': [],
}

amc_project_df = amc_project_df.set_index('project_id')

for k, v in project_nav_pm_dict.items():
    project_id = k
    date_dict = v
    amc = amc_project_df.loc[project_id]
    amc_id = amc['amc_id']
    amc_name = amc['amc_name']
    project_name = amc['project_name']
    
    management_style = amc['management_style']
    policy_desc = amc['policy_desc']
    fund_status = amc['fund_status']
    if management_style == 'PM':
        if project_id not in PM_SET50_proj_ids:
            continue
            
    for k, v in date_dict.items():
        date = k
        if v is None:
            continue
        project_nav_df_dict.get('amc_id').append(amc_id)
        project_nav_df_dict.get('amc_name').append(amc_name)
        project_nav_df_dict.get('project_id').append(project_id)
        project_nav_df_dict.get('project_name').append(project_name)
        project_nav_df_dict.get('management_style').append(management_style)
        project_nav_df_dict.get('policy_desc').append(policy_desc)
        project_nav_df_dict.get('fund_status').append(fund_status)
        project_nav_df_dict.get('nav_date').append(date)
        project_nav_df_dict.get('net_asset').append(v['net_asset'])
        project_nav_df_dict.get('last_val').append(v['last_val'])
        project_nav_df_dict.get('previous_val').append(v['previous_val'])
        
amc_project_df = amc_project_df.reset_index()

project_nav_pm_df = pd.DataFrame(project_nav_df_dict)
project_nav_pm_df.to_csv('csv/project_nav_pm.csv', index=False)
print(project_nav_pm_df[project_nav_pm_df['management_style'] == 'PM']['project_id'].unique())
project_nav_pm_df

['M0425_2559' 'M0575_2560' 'M0155_2547' 'M0580_2560' 'M0653_2559'
 'M0691_2559' 'M0259_2550' 'M0227_2562' 'M0292_2562' 'M0001_2552'
 'M0209_2548' 'M0738_2556' 'M0597_2552' 'M0787_2554' 'M0132_2557'
 'M0712_2547' 'M0943_2554']


,amc_id,amc_name,project_id,project_name,management_style,policy_desc,fund_status,nav_date,net_asset,last_val,previous_val
0,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0425_2559,K SET50 LTF,PM,ตราสารทุน,RG,2019-10-02,1.344928e+09,12.1637,12.2522
1,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0425_2559,K SET50 LTF,PM,ตราสารทุน,RG,2020-02-14,1.825326e+09,11.6247,11.6780
2,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0425_2559,K SET50 LTF,PM,ตราสารทุน,RG,2019-10-17,1.393835e+09,12.3953,12.4031
3,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0425_2559,K SET50 LTF,PM,ตราสารทุน,RG,2020-02-12,1.846873e+09,11.7457,11.6100
4,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0425_2559,K SET50 LTF,PM,ตราสารทุน,RG,2020-02-28,1.611012e+09,10.2422,10.7017
...,...,...,...,...,...,...,...,...,...,...,...
23131,C0000000714,THANACHART FUND MANAGEMENT COMPANY LIMITED,M0943_2554,Thanachart SET50 Fund,PM,ตราสารแห่งทุน,RG,2019-01-17,1.542363e+09,14.8239,14.7954
23132,C0000000714,THANACHART FUND MANAGEMENT COMPANY LIMITED,M0943_2554,Thanachart SET50 Fund,PM,ตราสารแห่งทุน,RG,2018-11-14,1.375616e+09,15.3177,15.3889
23133,C0000000714,THANACHART FUND MANAGEMENT COMPANY LIMITED,M0943_2554,Thanachart SET50 Fund,PM,ตราสารแห่งทุน,RG,2018-07-09,9.976123e+08,14.8514,14.7699
23134,C0000000714,THANACHART FUND MANAGEMENT COMPANY LIMITED,M0943_2554,Thanachart SET50 Fund,PM,ตราสารแห่งทุน,RG,2019-03-22,1.652738e+09,15.5105,15.3507


In [12]:
project_fund_full_port_df_dict = {
    'amc_id': [],
#     'amc_name': [],
    'project_id': [],
#     'project_name': [],
    'assetliab_code': [],
    'period': [],
    'market_value': [],
    'percent_nav': [],
}

amc_project_df = amc_project_df.set_index('project_id')

for k, v in project_fund_full_port_dict.items():
    project_id = k
    date_dict = v
    amc = amc_project_df.loc[project_id]
    amc_id = amc['amc_id']
    amc_name = amc['amc_name']
    project_name = amc['project_name']
    for k, v in date_dict.items():
        date = k
        if v is None:
            continue
        for v in v:
            project_fund_full_port_df_dict.get('amc_id').append(amc_id)
    #         project_fund_full_port_df_dict.get('amc_name').append(amc_name)
            project_fund_full_port_df_dict.get('project_id').append(project_id)
    #         project_fund_full_port_df_dict.get('project_name').append(project_name)
            project_fund_full_port_df_dict.get('assetliab_code').append(v['assetliab_code'])
            project_fund_full_port_df_dict.get('period').append(v['period'])
            project_fund_full_port_df_dict.get('market_value').append(v['market_value'])
            project_fund_full_port_df_dict.get('percent_nav').append(v['persent_nav'])
        
amc_project_df = amc_project_df.reset_index()

project_fund_full_port_df = pd.DataFrame(project_fund_full_port_df_dict)
project_fund_full_port_df.to_csv('csv/project_fund_full_port.csv', index=False)
project_fund_full_port_df

,amc_id,project_id,assetliab_code,period,market_value,percent_nav
0,C0000000709,M0108_2545,101,201604,4.514685e+08,82.25837
1,C0000000709,M0108_2545,103,201604,1.805229e+07,3.28916
2,C0000000709,M0108_2545,110,201604,1.611480e+06,0.29361
3,C0000000709,M0108_2545,201,201604,2.850471e+07,5.19361
4,C0000000709,M0108_2545,206,201604,4.858967e+07,8.85313
...,...,...,...,...,...,...
44997,C0000028866,M0246_2562,108,201912,7.140182e+07,97.96630
44998,C0000028866,M0246_2562,201,201912,9.634023e+06,13.21828
44999,C0000028866,M0246_2562,401,201912,1.922739e+05,0.26381
45000,C0000028866,M0246_2562,700,201912,-8.344053e+06,-11.44839


In [13]:
project_fund_full_port_pivot = project_fund_full_port_df.pivot_table(
    index=['amc_id', 'project_id', 'period'],
    columns='assetliab_code',
    values=['market_value', 'percent_nav'],
    aggfunc='first'
).reset_index()

project_fund_full_port_pivot.to_csv('csv/project_fund_full_port_pivot.csv')
project_fund_full_port_pivot

amc_id  project_id  period market_value      \
assetliab_code                                           101 103   
0               C0000000021  M0008_2537  201802  422236629.0 NaN   
1               C0000000021  M0008_2537  201803  414728151.7 NaN   
2               C0000000021  M0008_2537  201804  417384909.3 NaN   
3               C0000000021  M0008_2537  201805  411274772.1 NaN   
4               C0000000021  M0008_2537  201806  377315105.6 NaN   
...                     ...         ...     ...          ...  ..   
8815            C0000028866  M0354_2562  201910          NaN NaN   
8816            C0000028866  M0354_2562  201911          NaN NaN   
8817            C0000028866  M0354_2562  201912          NaN NaN   
8818            C0000028866  M0354_2562  202001          NaN NaN   
8819            C0000028866  M0354_2562  202002          NaN NaN   

                                                               ...  \
assetliab_code           108 110          201 206         401  ...   
0                        NaN NaN  18435672.92 NaN         NaN  ...   
1                        NaN NaN  10546755.56 NaN         NaN  ...   
2                        NaN NaN  13543102.56 NaN         NaN  ...   
3                        NaN NaN  10961497.73 NaN         NaN  ...   
4                        NaN NaN  10709175.39 NaN         NaN  ...   
...                      ...  ..          ...  ..         ...  ...   
8815            1.176133e+08 NaN   3296151.11 NaN   691475.32  ...   
8816            1.137485e+08 NaN   6422294.66 NaN   188694.17  ...   
8817            5.903879e+07 NaN  48468320.64 NaN   556869.04  ...   
8818            4.450693e+07 NaN   3227813.66 NaN  -950764.74  ...   
8819            3.829097e+07 NaN   1054537.51 NaN -1446945.83  ...   

                             percent_nav                                   \
assetliab_code           903         101 103        108 110       201 206   
0               4.406124e+08    95.82949 NaN        NaN NaN   4.18410 NaN   
1               4.271061e+08    97.10191 NaN        NaN NaN   2.46935 NaN   
2               4.308865e+08    96.86656 NaN        NaN NaN   3.14308 NaN   
3               4.193795e+08    98.06746 NaN        NaN NaN   2.61374 NaN   
4               3.913897e+08    96.40394 NaN        NaN NaN   2.73619 NaN   
...                      ...         ...  ..        ...  ..       ...  ..   
8815            1.224220e+08         NaN NaN   96.07207 NaN   2.69245 NaN   
8816            1.234885e+08         NaN NaN   92.11259 NaN   5.20072 NaN   
8817            6.120935e+07         NaN NaN   96.45387 NaN  79.18450 NaN   
8818            4.501301e+07         NaN NaN   98.87572 NaN   7.17085 NaN   
8819            3.775546e+07         NaN NaN  101.41837 NaN   2.79307 NaN   

                                          
assetliab_code      401       700    903  
0                   NaN  -0.01359  100.0  
1                   NaN   0.42873  100.0  
2                   NaN  -0.00963  100.0  
3                   NaN  -0.68120  100.0  
4                   NaN   0.85986  100.0  
...                 ...       ...    ...  
8815            0.56483   0.67065  100.0  
8816            0.15280   2.53388  100.0  
8817            0.90978 -76.54815  100.0  
8818           -2.11220  -3.93436  100.0  
8819           -3.83241  -0.37903  100.0  

[8820 rows x 21 columns]

In [14]:
project_fund_top5_df_dict = {
    'amc_id': [],
#     'amc_name': [],
    'project_id': [],
#     'project_name': [],
    'assetseq': [],
    'secur_name': [],
    'secur_abbr_name': [],
    'period': [],
    'secur_invest_size': [],
}

amc_project_df = amc_project_df.set_index('project_id')

for k, v in project_fund_top5_dict.items():
    project_id = k
    date_dict = v
    amc = amc_project_df.loc[project_id]
    amc_id = amc['amc_id']
    amc_name = amc['amc_name']
    project_name = amc['project_name']
    for k, v in date_dict.items():
        date = k
        if v is None:
            continue
        for v in v:
            project_fund_top5_df_dict.get('amc_id').append(amc_id)
    #         project_fund_top5_df_dict.get('amc_name').append(amc_name)
            project_fund_top5_df_dict.get('project_id').append(project_id)
    #         project_fund_top5_df_dict.get('project_name').append(project_name)
            project_fund_top5_df_dict.get('assetseq').append(v['assetseq'])
            project_fund_top5_df_dict.get('secur_name').append(v['secur_name'])
            project_fund_top5_df_dict.get('secur_abbr_name').append(v['secur_abbr_name'])
            project_fund_top5_df_dict.get('period').append(v['period'])
            project_fund_top5_df_dict.get('secur_invest_size').append(v['secur_invest_size'])
        
amc_project_df = amc_project_df.reset_index()

project_fund_top5_df = pd.DataFrame(project_fund_top5_df_dict)
project_fund_top5_df.to_csv('csv/project_fund_top5.csv', index=False)
project_fund_top5_df

,amc_id,project_id,assetseq,secur_name,secur_abbr_name,period,secur_invest_size
0,C0000000021,M0705_2548,1,BANK OF THAILAND,CB19704B,201906,7.62
1,C0000000021,M0705_2548,2,BANK OF THAILAND,CB19711B,201906,7.14
2,C0000000021,M0705_2548,3,BANK OF THAILAND,CB19725B,201906,6.66
3,C0000000021,M0705_2548,4,BANK FOR AGRICULTURE AND AGRICULTURAL COOPERAT...,GGLB197A,201906,4.80
4,C0000000021,M0705_2548,5,Kasikorn Bank Plc.,SAVKBANK.I,201906,4.76
...,...,...,...,...,...,...,...
42830,C0000028866,M0354_2562,1,"JP MORGAN CHASE BANK, LONDON",USD WE-GA5M,201912,77.84
42831,C0000028866,M0354_2562,2,SSGA FUNDS MANAGEMENT INC,SPY US,201912,40.95
42832,C0000028866,M0354_2562,3,INVESCO POWERSHARES CAPITAL MANAGEMENT LLC,QQQ US,201912,21.86
42833,C0000028866,M0354_2562,4,BLACKROCK FUND ADVISORS,FEZ US,201912,9.96


In [15]:
project_fund_top5_pivot = project_fund_top5_df.pivot_table(
    index=['amc_id', 'project_id', 'period'],
    columns='assetseq',
    values=['secur_name', 'secur_invest_size'],
    aggfunc='first'
).reset_index()

project_fund_top5_pivot.to_csv('csv/project_fund_top5_pivot.csv')
project_fund_top5_pivot

amc_id  project_id  period secur_invest_size                \
assetseq                                                  1      2      3   
0         C0000000021  M0008_2537  201901              7.06   6.87   6.37   
1         C0000000021  M0008_2537  201902              7.91   6.43   5.64   
2         C0000000021  M0008_2537  201903              7.90   7.05   6.23   
3         C0000000021  M0008_2537  201904              8.98   8.18   7.97   
4         C0000000021  M0008_2537  201905              9.87   7.43   6.71   
...               ...         ...     ...               ...    ...    ...   
8718      C0000028866  M0354_2562  201910             26.17  23.15   9.72   
8719      C0000028866  M0354_2562  201911             33.08  18.07  16.58   
8720      C0000028866  M0354_2562  201912             77.84  40.95  21.86   
8721      C0000028866  M0354_2562  202001             35.59  24.23  10.31   
8722      C0000028866  M0354_2562  202002             39.70  29.20  27.58   

                                                secur_name  \
assetseq      4     5                                    1   
0          5.71  5.05           PTT PUBLIC COMPANY LIMITED   
1          4.91  4.91           PTT PUBLIC COMPANY LIMITED   
2          5.18  4.99           PTT PUBLIC COMPANY LIMITED   
3          5.25  5.14           PTT PUBLIC COMPANY LIMITED   
4          6.02  5.46                    BANGKOK BANK PLC.   
...         ...   ...                                  ...   
8718       7.92  7.37  State Street Bank and Trust Company   
8719      13.33  5.74                               SSGA12   
8720       9.96  9.57         JP MORGAN CHASE BANK, LONDON   
8721       9.95  6.89             SSGA FUND MANAGEMENT INC   
8722       4.93  2.42            SSGA FUNDS MANAGEMENT INC   

                                                       \
assetseq                                            2   
0         AIRPORTS OF THAILAND PUBLIC COMPANY LIMITED   
1         AIRPORTS OF THAILAND PUBLIC COMPANY LIMITED   
2         AIRPORTS OF THAILAND PUBLIC COMPANY LIMITED   
3                                   BANGKOK BANK PLC.   
4                          PTT PUBLIC COMPANY LIMITED   
...                                               ...   
8718                                  BlackRock, Inc.   
8719       Invesco PowerShares Capital Management LLC   
8720                        SSGA FUNDS MANAGEMENT INC   
8721       INVESCO POWERSHARES CAPITAL MANAGEMENT LLC   
8722                                  BlackRock, Inc.   

                                                       \
assetseq                                            3   
0                                   BANGKOK BANK PLC.   
1                 KASIKORNBANK PUBLIC COMPANY LIMITED   
2                                   BANGKOK BANK PLC.   
3         AIRPORTS OF THAILAND PUBLIC COMPANY LIMITED   
4         AIRPORTS OF THAILAND PUBLIC COMPANY LIMITED   
...                                               ...   
8718                                          681194Z   
8719                                  BlackRock, Inc.   
8720       INVESCO POWERSHARES CAPITAL MANAGEMENT LLC   
8721              STATE STREET BANK AND TRUST COMPANY   
8722       Invesco PowerShares Capital Management LLC   

                                                        \
assetseq                                             4   
0                  KASIKORNBANK PUBLIC COMPANY LIMITED   
1                  BANGKOK BANK PUBLIC COMPANY LIMITED   
2         ADVANCED INFO SERVICE PUBLIC COMPANY LIMITED   
3         ADVANCED INFO SERVICE PUBLIC COMPANY LIMITED   
4         ADVANCED INFO SERVICE PUBLIC COMPANY LIMITED   
...                                                ...   
8718                         SSgA Fund Management Inc.   
8719                         SSgA Fund Management Inc.   
8720                           BLACKROCK FUND ADVISORS   
8721                              ETF Management Group   
8722               STATE S

In [4]:
set50_df = pd.read_csv('SET_50_Historical_Data.csv')
set50_df

,Date,Price,Open,High,Low,Vol.,Change %
0,"Apr 09, 2020",816.47,825.30,842.10,809.88,2.68B,0.17%
1,"Apr 08, 2020",815.05,814.38,823.28,805.74,2.61B,-1.26%
2,"Apr 07, 2020",825.42,804.69,830.44,793.47,3.11B,7.66%
3,"Apr 03, 2020",766.67,777.96,782.72,760.30,3.79B,-0.15%
4,"Apr 02, 2020",767.79,747.35,772.59,741.04,4.07B,3.69%
...,...,...,...,...,...,...,...
2115,"Aug 22, 2011",741.98,742.08,742.53,729.07,921.54K,-0.12%
2116,"Aug 19, 2011",742.86,746.31,751.14,740.64,1.22M,-2.10%
2117,"Aug 18, 2011",758.83,765.44,768.51,757.87,1.09M,-0.26%
2118,"Aug 17, 2011",760.83,755.34,762.55,754.04,1.06M,1.60%


In [17]:
import datetime

def format_date_string(date_string):
    return datetime.datetime.strptime(date_string, '%b %d, %Y').strftime('%Y-%m-%d')
    
date_col = set50_df['Date']
set50_df['date_formatted'] = list(map(format_date_string, date_col))
set50_df.to_csv('csv/SET_50_Historical_Data_Formatted.csv', index=False)
set50_df

,Date,Price,Open,High,Low,Vol.,Change %,date_formatted
0,"Apr 09, 2020",816.47,825.30,842.10,809.88,2.68B,0.17%,2020-04-09
1,"Apr 08, 2020",815.05,814.38,823.28,805.74,2.61B,-1.26%,2020-04-08
2,"Apr 07, 2020",825.42,804.69,830.44,793.47,3.11B,7.66%,2020-04-07
3,"Apr 03, 2020",766.67,777.96,782.72,760.30,3.79B,-0.15%,2020-04-03
4,"Apr 02, 2020",767.79,747.35,772.59,741.04,4.07B,3.69%,2020-04-02
...,...,...,...,...,...,...,...,...
2115,"Aug 22, 2011",741.98,742.08,742.53,729.07,921.54K,-0.12%,2011-08-22
2116,"Aug 19, 2011",742.86,746.31,751.14,740.64,1.22M,-2.10%,2011-08-19
2117,"Aug 18, 2011",758.83,765.44,768.51,757.87,1.09M,-0.26%,2011-08-18
2118,"Aug 17, 2011",760.83,755.34,762.55,754.04,1.06M,1.60%,2011-08-17
